In [142]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 1000)

# Steps
1. empty hand with 1.0 chance
+ or other hands: determine whih step to take next (e.g. plyer finished, split hand etc.)
2. deal random cards with chances
+ combine same hands (5,8 and 8,5)
+ finish blackjacks
3. get all possible moves for player, keep the drawn cards saved
+ vectorized adding (with diag)
+ [FALSE] aggregate bust hands (e.g. a 18 only needs 11,2,3 and 4+ can be immediately seen as a bust hand)
4. get all possible moves for dealer
+ aggregate deck and dealer info as input: same deck info means same dealer possibilities
+ after getting each possible score from deck info, calculate win lose and push ratios for each player hand
+ (same deck info mean same hand? then even better)
5. for each drawn hand starting from the back, determine the best move (hit or stand)
+ save best moves for possible combinations, use them on all possible instead of recaculating
+ dont include obvious cases (e.g. stand on 21, hit under 12)
6. determine expected return for hands as far back as you like (start of game, dealt hands, dealer plays)

In [143]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["score", "cards", "aces"]
ACTIVE_COLUMNS = ["player_"+colname for colname in HAND_COLUMNS] + ["dealer_"+colname for colname in HAND_COLUMNS]
GAME_COLUMNS = ["chance"] + ACTIVE_COLUMNS + DECK_COLUMNS

# SETUP

In [144]:
def get_deck(number_decks=6):
    """get a deck with the specified number of decks (default: 6)"""
    
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [145]:
def order_game_columns(game):
    """order the columns of the game dataframe"""

    return game[GAME_COLUMNS + [colname for colname in game.columns if colname not in GAME_COLUMNS]]

In [146]:
def player_draws_single(game, value, probabilistic=True, update_deck=True):
    """update the game with the player drawing a card with the specified value"""

    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["player_score"] += value
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game["player_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    return next_game

In [147]:
def dealer_draws_single(game, value, probabilistic=True, update_deck=True):
    """update the game with the dealer drawing a card with the specified value"""

    assert np.all(game["dealer_score"] >= 0) and np.all(game["dealer_score"] < 17), f"there are finished hands, but dealer tried to add card {value}. hands:\n{game}"

    next_game = game.copy()

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        next_game["chance"] *= (next_game[f"{value}"]/next_game[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        next_game[f"{value}"] -= 1

    # updating score
    next_game["dealer_score"] += value
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game["dealer_aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -1
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1 

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    return next_game

In [148]:
def get_game(deck=None, player_cards=[], dealer_cards=[], update_deck=True):
    """get a game with the specified deck, player cards and dealer cards"""

    if deck is None:
        deck = get_deck()
    
    game = pd.DataFrame(
        [1.0] + [0, 0, 0]*2 + deck.values.flatten().tolist(), 
        index=GAME_COLUMNS
    ).T.astype(int).astype({"chance":float})

    for player_card_value in player_cards:
        game = player_draws_single(game, player_card_value, probabilistic=False, update_deck=update_deck)
    
    for dealer_card_value in dealer_cards:
        game = dealer_draws_single(game, dealer_card_value, probabilistic=False, update_deck=update_deck)
    
    return game

# PLAYER

In [149]:
def player_draws_once(game, keep_drawn_column=False):
    """get updated game when player hits once, throws error for finished hands"""

    assert np.all(game["dealer_cards"] != 0), f"dealer must have one card showing. hands: {game}"    
    assert np.all(game["player_score"] >= 0) and np.all(game["player_score"] != 22), f"there are finished hands, but player tried to hit. hands:\n{game}"
    assert game[DECK_COLUMNS].sum(axis=1).max() == game[DECK_COLUMNS].sum(axis=1).min(), f"there are decks with different number of cards. hands:\n{game}"
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["player_score"] += next_game["drawn"]
    # updating cards
    next_game["player_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "player_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["player_score"] > 21) & (next_game["player_aces"] > 0)
    next_game.loc[acenecessary,"player_score"] -= 10
    next_game.loc[acenecessary,"player_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["player_score"] > 21, "player_score"] = -2

    # set blackjack to 22
    next_game.loc[(next_game["player_score"] == 21) & (next_game["player_cards"] == 2), "player_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # aggregate same hands
    cols = next_game.columns.tolist()
    cols.remove("chance")
    next_game = next_game.groupby(cols).agg({'chance': 'sum'}).reset_index()
    # NOTE: is drawn column is kept, the hands will not be grouped (useful for player decisions)

    # sort columns
    next_game = order_game_columns(next_game)

    return next_game.sort_values("player_score")


In [150]:
def player_plays_all_possible(game):
    is_playable = (game["player_score"] >= 0) & (game["player_score"] != 22) & (game["player_score"] != 21)
    playable = game[is_playable].copy()
    final = game.copy()

    # print("calculating all possible moves...")
    while len(playable) > 0:
        next_game = player_draws_once(playable, keep_drawn_column=True)
        drawnsum = len([i for i in next_game.columns.tolist() if "drawn" in i])
        next_game = next_game.rename(columns={"drawn":"drawn_"+str(drawnsum)})
        final["drawn_"+str(drawnsum)] = np.zeros(len(final)).astype(int)
        
        final = pd.concat((final, next_game), axis=0, ignore_index=True)

        is_playable = (next_game["player_score"] >= 0) & (next_game["player_score"] != 22) & (next_game["player_score"] != 21)
        playable = next_game[is_playable].copy()
    
    return final

# DEALER

In [151]:
def dealer_draws_once(game, keep_drawn_column=False):
    """get updated game when dealer hits once"""
    
    # add possible cards to draw
    next_game = pd.merge(game, pd.DataFrame(np.arange(2,12), columns=["drawn"]), how="cross")

    # helper diag matrix
    diag = np.tile(np.diag(np.ones(10)).astype(bool), (len(game),1))

    # update drawing chance
    next_game["chance"] *= next_game[DECK_COLUMNS].values[diag] / next_game[DECK_COLUMNS].sum(axis=1).values

    # TODO: 0 boundary check, what if there are no cards of that type left?
    # currently works, because the chance is 0 anyways

    # update deck info
    next_game.loc[:,DECK_COLUMNS] -= diag.astype(int)

    # updating score
    next_game["dealer_score"] += next_game["drawn"]
    # updating cards
    next_game["dealer_cards"] += 1
    # updating aces
    next_game.loc[next_game["drawn"]== 11, "dealer_aces"] += 1

    # if necessary, using ace as 1 instead of 11
    acenecessary = (next_game["dealer_score"] > 21) & (next_game["dealer_aces"] > 0)
    next_game.loc[acenecessary,"dealer_score"] -= 10
    next_game.loc[acenecessary,"dealer_aces"] -= 1

    # set bust to -2
    next_game.loc[next_game["dealer_score"] > 21, "dealer_score"] = -1

    # set blackjack to 22
    next_game.loc[(next_game["dealer_score"] == 21) & (next_game["dealer_cards"] == 2), "dealer_score"] = 22

    # drop drawn column
    if not keep_drawn_column:
        next_game = next_game.drop(columns=["drawn"])

    # group and reorder
    next_game = next_game.groupby(next_game.columns.tolist()[1:]).agg({'chance': 'sum'}).reset_index()
    next_game = order_game_columns(next_game)

    return next_game.sort_values("dealer_score")


In [152]:
def get_dealer_outcomes(game):
    """get outcomes for dealer play, grouped by original deck info"""
    
    assert set(DECK_COLUMNS).issubset(set(game.columns.tolist())), "Deck Info not in columns"
    assert set(["player_score","player_cards","player_aces","dealer_score","dealer_aces","dealer_cards","chance"]).issubset(game.columns.tolist()), "not all info in columns"
    assert "drawn" not in game.columns, f"drawn column already exists. hands:\n{game}"

    # keep original deck info
    for deck_column in DECK_COLUMNS:
        game["orig_"+deck_column] = game[deck_column]

    is_playable = (game["dealer_score"] >= 0) & (game["dealer_score"] < 17)
    final = game[~is_playable]
    playable = game[is_playable]

    while len(playable) > 0:
        next_game = dealer_draws_once(playable)

        is_playable = (next_game["dealer_score"] < 17) & (next_game["dealer_score"] >= 0)
        # finished hands are taken out of the game
        final = pd.concat((final, next_game[~is_playable]), axis=0, ignore_index=True)
        # playable hands
        playable = next_game[is_playable].copy()

    grouped = final.groupby(["orig_"+col for col in DECK_COLUMNS] + ["player_score","player_cards","player_aces","dealer_score","dealer_aces","dealer_cards"]).agg({'chance':'sum'}).reset_index()

    
    return grouped

In [153]:
def dealer_plays(game, verbose=False):
    """extends game rows by possible dealer outcomes, adds column "expected_return" """

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # get distinct decks
    distinct_decks = game.copy().groupby(["player_score","player_cards","player_aces","dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS).agg({"chance":"min"}).reset_index()
    distinct_decks2 = game.copy().groupby(["player_score","player_cards","player_aces","dealer_score","dealer_cards","dealer_aces"] + DECK_COLUMNS).agg({"chance":"max"}).reset_index()

    assert np.all(np.isclose(distinct_decks, distinct_decks2)), "chances are not the same for same decks"

    if verbose:
        print("game\n", game)
        print(f"Simplified from {len(game)} to {len(distinct_decks)} games")
        print("distinct decks\n", distinct_decks)

    distinct_decks = order_game_columns(distinct_decks)

    dealer_scores = get_dealer_outcomes(distinct_decks)
    dealer_scores = get_return_column(dealer_scores)

    dealer_scores["expected_return_normed"] = dealer_scores["return"] * dealer_scores["chance"]

    dealer_scores_grouped = dealer_scores.groupby(["orig_"+c for c in DECK_COLUMNS]).agg({"expected_return_normed":"sum","chance":"sum"}).reset_index()

    if verbose:
        print("dealer scores grouped\n", dealer_scores_grouped)

    game = game.rename(columns={"chance":"chance_orig"})
    
    # merge back to original game
    game = pd.merge(game, dealer_scores_grouped.loc[:, ["orig_"+s for s in DECK_COLUMNS] + ["expected_return_normed","chance"]], left_on=DECK_COLUMNS, right_on=["orig_"+s for s in DECK_COLUMNS], how="left")

    assert np.all(np.isclose(game["chance_orig"],game["chance"]))
    game["expected_return"] = game["chance_orig"] * (game["expected_return_normed"] / game["chance"])

    return game


# RETURNS

In [154]:
def get_expected_return_standing(game, verbose=False):
    """add column "expected return" to game,
    player has minimum of 2 cards, dealer has 1 upcard
    assumes perfect play"""

    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"
    assert np.all(game["dealer_cards"] == 1), f"dealer must have one card showing. hands:\n{game}"

    # bust hands
    is_bust = game["player_score"] == -2
    finished = game[is_bust].copy()
    playable = game[~is_bust].copy()

    if verbose:
        print("Calculating possible returns for each game...")
        print(f"Bust games: {len(finished) / len(game) * 100:.2f}%")

    dealer_outcomes = dealer_plays(playable, verbose)

    dealer_outcomes = dealer_outcomes.drop(columns=["chance_orig","expected_return_normed"])

    if verbose:
        print("dealer outcomes\n", dealer_outcomes[[col for col in dealer_outcomes.columns if ("orig_" not in col)]])

    finished["expected_return"] = -finished["chance"]
    
    final = pd.concat((finished, dealer_outcomes[[col for col in dealer_outcomes.columns if ("orig_" not in col)]]), axis=0, ignore_index=True)

    return final


In [155]:
def get_return_column(game):
    """add "return" column to game dataframe, denoting the return of the game"""

    assert np.all((game["dealer_score"] < 0) | (game["dealer_score"] >= 17)), "dealer has not finished yet"
    assert np.all(game["player_cards"] >= 2), f"player has not played 2 cards yet, played minimum of {game['player_cards'].min()} cards"

    game["return"] = pd.Series(np.zeros(len(game)))

    game.loc[(game["player_score"] == 22) & (game["dealer_score"] != 22), "return"] = 1.5
    game.loc[(game["player_score"] > game["dealer_score"]) & (game["player_score"] != 22), "return"] = 1
    game.loc[game["player_score"] == game["dealer_score"], "return"] = 0
    game.loc[game["player_score"] < game["dealer_score"], "return"] = -1

    return game

In [255]:
def get_expected_return_best(game, verbose=False):
    """add column "expected_return" to game dataframe
    assumes best play"""

    t1 = time.time()

    stand_return = get_expected_return_standing(game)

    if np.all(game["player_score"] == 22):
        return stand_return

    final = player_plays_all_possible(game)
    final = get_expected_return_standing(final, verbose)

    print(final)

    drawn_columns = [i for i in final.columns.tolist() if "drawn" in i]

    # get double scores if player has 2 cards and can draw at least once 
    if np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" in drawn_columns:
        if verbose:
            print("double vals\n", final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), [col for col in final.columns if "drawn" not in col]+["drawn_1"]])
        double_return = final.loc[(final["drawn_1"] != 0) & (final["drawn_2"] == 0), "expected_return"].sum() * 2
    elif np.all(game["player_cards"] == 2) and "drawn_1" in drawn_columns and "drawn_2" not in drawn_columns:
        double_return = final.loc[(final["drawn_1"] != 0), "expected_return"].sum() * 2
    else:
        double_return = None

    for i in range(len(drawn_columns)):
        col = drawn_columns[len(drawn_columns)-i-1]
        cols = drawn_columns[:len(drawn_columns)-i-1]
        print(col)

        # get all games that can stand or hit in that column
        if len(cols) != 0:
            useable = (final[col] != 0) | ((final[col] == 0) & (final[cols[-1]] != 0) & (final["player_score"] > 0) & (final["player_score"] != 21))
        else:
            useable = (final[col] != 0) | ((final[col] == 0) & (final["player_score"] > 0))
        used = final[useable].copy()

        stand_column = used[col] == 0
        used["stand"] = stand_column
        # print("used\n", used.sort_values(cols+[col]))

        # group all games that hit
        grouped = used.groupby(cols + ["stand"]).agg({'chance': 'sum', "expected_return": "sum"}).reset_index()
        # print("grouped by cols and stand\n",grouped.sort_values(cols+["stand"]))

        standing_games = used[used["stand"]].copy().drop(columns=["expected_return",col,"stand"])
        print("standing_games\n", standing_games.sort_values(cols))
        
        # if there are no drawn columns left, the expected returns can be returned
        if len(cols) != 0:
            agg_cols = grouped.columns.tolist()
            agg_cols.remove("chance")
            best_options = grouped.groupby(cols).agg({"chance":"max","expected_return": "max" }).reset_index()
            print("best options\n", best_options.sort_values(cols))

            standing_games = pd.merge(standing_games, best_options.loc[:, cols + ["expected_return"]], on=cols, how="left")
            print("merged\n", standing_games.sort_values(cols))
        else:
            # this is for the last step, currently not working
            print(grouped)
            best_options = grouped.groupby("stand").agg({"chance":"max","expected_return": "max" }).reset_index()
            print("best options\n", best_options.sort_values("stand"))

            standing_games = pd.merge(standing_games, best_options.loc[:, ["stand","expected_return"]], on=cols, how="left")
            print("merged\n", standing_games.sort_values("stand"))
        
        # drop drawn column from unused games
        final = final[~useable].copy().drop(columns=[col])

        final = pd.concat([final, standing_games], axis=0, ignore_index=True)

        print(final)

    return final

# TESTING

In [263]:
game = get_game(None, [10,10],[11])
# game2 = get_game(None, [10,9],[11])
# game = pd.concat([game,game2], axis=0, ignore_index=True)
game["chance"] = game["chance"] / game["chance"].sum()
game

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11
0,1.0,20,2,0,11,1,1,24,24,24,24,24,24,24,24,94,23


In [264]:
# this works well, however the last step is currently not supported
# prints best options, I could just max() the expected_return but this iwll not work for multiple
# 
# next step is to make it work for n input games, I think for that I will need some kind of identifier so that the game knows which initial game to map it to...
# we will see
 
get_expected_return_best(game)

      chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11  drawn_1  expected_return
0   0.077670            -2             3            0            11             1            1  23  24  24  24  24  24  24  24  94  23        2        -0.077670
1   0.077670            -2             3            0            11             1            1  24  23  24  24  24  24  24  24  94  23        3        -0.077670
2   0.077670            -2             3            0            11             1            1  24  24  23  24  24  24  24  24  94  23        4        -0.077670
3   0.077670            -2             3            0            11             1            1  24  24  24  23  24  24  24  24  94  23        5        -0.077670
4   0.077670            -2             3            0            11             1            1  24  24  24  24  23  24  24  24  94  23        6        -0.077670
5   0.077670            -2        

IndexError: list index out of range

In [159]:
pos = player_plays_all_possible(game)
pos[pos["player_score"] > 0].sort_values("player_score")

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,drawn_1,drawn_2,drawn_3,drawn_4,drawn_5,drawn_6,drawn_7,drawn_8,drawn_9,drawn_10,drawn_11,drawn_12
0,1.000000e+00,8,2,0,11,1,1,24,23,24,23,24,24,24,24,96,23,0,0,0,0,0,0,0,0,0,0,0,0
1,7.766990e-02,10,3,0,11,1,1,23,23,24,23,24,24,24,24,96,23,2,0,0,0,0,0,0,0,0,0,0,0
2,7.443366e-02,11,3,0,11,1,1,24,22,24,23,24,24,24,24,96,23,3,0,0,0,0,0,0,0,0,0,0,0
41,5.558358e-03,12,4,0,11,1,1,24,22,24,23,24,24,24,24,96,22,11,3,0,0,0,0,0,0,0,0,0,0
40,5.558358e-03,12,4,0,11,1,1,24,22,24,23,24,24,24,24,96,22,3,11,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12432,1.556643e-07,21,8,0,11,1,1,24,21,23,23,24,24,24,24,96,20,3,11,11,11,4,3,0,0,0,0,0,0
12433,1.556643e-07,21,8,0,11,1,1,24,21,23,23,24,24,24,24,96,20,4,3,3,11,11,11,0,0,0,0,0,0
12434,1.556643e-07,21,8,0,11,1,1,24,21,23,23,24,24,24,24,96,20,4,3,11,3,11,11,0,0,0,0,0,0
12458,1.556643e-07,21,8,0,11,1,1,24,21,23,23,24,24,24,24,96,20,3,11,3,4,11,11,0,0,0,0,0,0


In [160]:
get_expected_return_standing(game)

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,expected_return
0,1.0,8,2,0,11,1,1,24,23,24,23,24,24,24,24,96,23,-0.76905


In [161]:
get_expected_return_best(game)

             chance  player_score  player_cards  player_aces  dealer_score  dealer_cards  dealer_aces   2   3   4   5   6   7   8   9  10  11  drawn_1  drawn_2  drawn_3  drawn_4  drawn_5  drawn_6  drawn_7  drawn_8  drawn_9  drawn_10  drawn_11  drawn_12  expected_return
0      2.420880e-02            -2             4            0            11             1            1  24  23  23  23  24  24  24  24  95  23        4       10        0        0        0        0        0        0        0         0         0         0    -2.420880e-02
1      9.582650e-02            -2             4            0            11             1            1  24  23  24  23  24  24  24  24  94  23       10       10        0        0        0        0        0        0        0         0         0         0    -9.582650e-02
2      2.420880e-02            -2             4            0            11             1            1  24  23  24  23  24  24  24  23  95  23       10        9        0        0        0    

,chance,player_score,player_cards,player_aces,dealer_score,dealer_cards,dealer_aces,2,3,4,5,6,7,8,9,10,11,drawn_1,drawn_2,drawn_3,drawn_4,drawn_5,drawn_6,drawn_7,drawn_8,drawn_9,drawn_10,drawn_11,expected_return
0,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,23.0,23.0,24.0,24.0,24.0,24.0,95.0,23.0,4,10,0,0,0,0,0,0,0,0,0,-2.420880e-02
1,9.582650e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,24.0,94.0,23.0,10,10,0,0,0,0,0,0,0,0,0,-9.582650e-02
2,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,23.0,95.0,23.0,10,9,0,0,0,0,0,0,0,0,0,-2.420880e-02
3,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,23.0,95.0,23.0,9,10,0,0,0,0,0,0,0,0,0,-2.420880e-02
4,2.420880e-02,-2.0,4.0,0.0,11.0,1.0,1.0,24.0,23.0,24.0,23.0,24.0,24.0,23.0,24.0,95.0,23.0,10,8,0,0,0,0,0,0,0,0,0,-2.420880e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25546,7.992345e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,2,11,11,2.442108e-14
25547,7.992345e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,11,2,11,2.442108e-14
25548,7.992345e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,11,11,2,2.442108e-14
25549,4.864906e-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,11,2,11,11,11,11,11,11,11,11,6.065802e-15
